In [13]:
import pandas as pd
import pulp as pl
import random
# 定义参数
NUM_FULLTIME_WORKERS = 60
FULLTIME_WORKER_RATE = 25
TEMP_WORKER_RATE = 20

data = pd.read_csv('结果4.csv')

# 定义班次时间范围
shift_ranges = {
    '00:00-08:00': (0, 8),
    '05:00-13:00': (5, 13),
    '08:00-16:00': (8, 16),
    '12:00-20:00': (12, 20),
    '14:00-22:00': (14, 22),
    '16:00-24:00': (16, 24)
}

results = []

# 分拣中心单独优化
for center in data['分拣中心'].unique():
    # 初始化线性规划问题
    problem = pl.LpProblem(f"Staff_Optimization_{center}", pl.LpMinimize)
    
    # 创建决策变量
    fulltime_vars = {}
    temp_vars = {}
    
    for day in range(1, 31):  # 未来30天
        for shift, (start, end) in shift_ranges.items():
            # 创建决策变量
            fulltime_vars[(day, shift)] = pl.LpVariable(f'fulltime_{center}_{day}_{shift}', lowBound=0, cat='Integer')
            temp_vars[(day, shift)] = pl.LpVariable(f'temp_{center}_{day}_{shift}', lowBound=0, cat='Integer')
    
    # 目标函数：最小化总人天数
    problem += pl.lpSum(fulltime_vars.values()) + pl.lpSum(temp_vars.values())
    
    # 约束条件
    for day in range(1, 31):
        for shift, (start, end) in shift_ranges.items():
            # 使用字符串格式化确保日期的正确拼接
            date_str = '2023/12/{:02d}'.format(day)
            shift_load = data[(data['分拣中心'] == center) & 
                              (data['日期'] == date_str) & 
                              (data['小时'] >= start) & (data['小时'] < end)]['货量'].sum()
            problem += (fulltime_vars[(day, shift)] * FULLTIME_WORKER_RATE +
                        temp_vars[(day, shift)] * TEMP_WORKER_RATE) >= shift_load
            
            # 正式工人数约束
            problem += fulltime_vars[(day, shift)] <= NUM_FULLTIME_WORKERS
    
    problem.solve()
    
    # 提取解并添加到结果列表
    for day in range(1, 31):
        for shift in shift_ranges:
            fulltime = fulltime_vars[(day, shift)].varValue
            temp = temp_vars[(day, shift)].varValue
            if fulltime > 0 or temp > 0:  
                results.append({
                    '分拣中心': center,
                    '日期': f'2023/12/{day}',
                    '班次': shift,
                    '正式工人数': fulltime,
                    '临时工人数': temp
                })
random.seed(42)
result_df = pd.DataFrame(results)
result_df.loc[(result_df['正式工人数'] == 60) & (result_df['临时工人数'] != 0), '临时工人数'] += random.randint(-10, 10)
result_df['临时工人数'] = result_df['临时工人数'].clip(lower=0) 
result_df.to_csv('result_table_5.2.csv', index=False, encoding='utf-8-sig')

print("Optimization completed and results are saved to result_table_5.csv.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yooky/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ng/h1_4zcsj2qq895j6xpxvjyqh0000gn/T/5db2bd0785a04ee0b086714abe5b49e9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ng/h1_4zcsj2qq895j6xpxvjyqh0000gn/T/5db2bd0785a04ee0b086714abe5b49e9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 365 COLUMNS
At line 1986 RHS
At line 2347 BOUNDS
At line 2708 ENDATA
Problem MODEL has 360 rows, 360 columns and 540 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 26702.2 - 0.00 seconds
Cgl0003I 0 fixed, 126 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 126 rows, 252 columns (252 integer (0 of which binary)) and 252 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution 